In [1]:
import warnings
warnings.filterwarnings('ignore')

from pymongo import MongoClient
import pprint

import copy
import pandas as pd

import requests

from bs4 import BeautifulSoup

### I'm gonna practice on the football one... let's go

Quick note... I think my football url was off in my csv... it should be 'football-soccer-player' rather than 'football-players'.

In [2]:
football_url = 'https://famouspeople.astro-seek.com/filter_occupation/?narozeni_stat=&povolani=football-soccer-player&narozeni_stat=&pohlavi=&razeni='
r = requests.get(football_url)

client = MongoClient('localhost', 27017)
db = client.metroid
pages = db.pages

pages.insert_one({'html': r.content})

soup = BeautifulSoup(r.content, 'html')

In [3]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   ASTRO-DB FILTER  -
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="en" http-equiv="content-language"/>
  <meta content="text/javascript" http-equiv="content-script-type"/>
  <meta content="text/css" http-equiv="content-style-type"/>
  <meta content="index,follow" name="robots"/>
  <meta content="ASTRO-DB FILTER  - " name="description"/>
  <meta content="ASTRO-DB, FILTER, , , astro-seek, astroseek, horoscopes, charts, signs, zodiac, numerology, birth" name="keywords"/>
  <meta content="Astro-Seek.com © 2016 - 2020" name="copyright"/>
  <meta content="NdrTyPCwuaPr_26Quymem-jlfZJo4CKw8JYkkc_fsao" name="google-site-verification"/>
  <meta content="Qaq5_iQ2vq2zsHrPOaZT3M-POCwZCeDJbHhKApKu-X0" name="google-site-verification"/>
  <meta content="DE

In [4]:
div = soup.find('div', {'class': 'obsah-vnitrek', 'style': 'padding-top: 0px; '})
#<div class="obsah-vnitrek" style="padding-top: 0px; ">
#<h3>Football Players <span>- Celebrities (1522)</span></h3>
header = div.find('h3')

In [8]:
s = header.text.strip()
number_list = [_ for _ in s if _.isdigit()]
number = int(''.join(number_list))
print(number)

1522


In [11]:
from math import ceil

In [12]:
number_of_pages = ceil(number / 200)
print(number_of_pages)

8


In [16]:
# https://famouspeople.astro-seek.com/filter_occupation/?narozeni_stat=&povolani=football-soccer-player&narozeni_stat=&pohlavi=&razeni=
# https://famouspeople.astro-seek.com/filter_occupation_200/?povolani=football-soccer-player&pohlavi=&razeni=
# https://famouspeople.astro-seek.com/filter_occupation_400/?povolani=football-soccer-player&pohlavi=&razeni=
# 
# 
# 
# 
# https://famouspeople.astro-seek.com/filter_occupation_1400/?povolani=football-soccer-player&pohlavi=&razeni=

the_url_beginning = 'https://famouspeople.astro-seek.com/filter_occupation'
the_url_filter = ''
the_url_middle = '/?povolani='
the_url_occupation = 'football-soccer-player'
#^the url occupation should come from csv (the name column).
the_url_ending = '&pohlavi=&razeni='

url_stem_lst = ['_' + str(200 * _) for _ in range(1,number_of_pages)]
url_lst = [the_url_beginning + _ + the_url_middle + the_url_occupation + the_url_ending for _ in url_stem_lst]
#print(url_lst)